Bachelor thesis - Richard Borschke - 7337876 - University of Cologne - A spatio-temporal analysis of usage patterns in free-floating shared mobility


# Data Preparation

This notebook contains the preprocessing of data, needed for analysis. Following steps are performed:
* Data loading of all provided trip files
* Data splitting into car, bicycle and scooter set
* Data cleaning of unusable trips and variables
* Data preparing of variables


time (18.11 to 29.2  +  05.11.)

data set misses 2 days (16. and 15. 01.)

car misses 4 days (06. to 09.12.)

### Imports

In [1]:
import pandas as pd
import glob
import numpy as np

### Methods

In [2]:
# load full data (all modes) based on selected months
def load_data(months):
    first_read = True
    for month in months:
        data_month = pd.concat([pd.read_csv(file) for file in glob.glob('Data/Trip_{}/*_trip.csv'.format(month))], 
                               ignore_index = True)
        if (first_read):
            data_full = data_month
            first_read = False
        else:
            data_full = pd.concat([data_full, data_month], ignore_index=True)
    return data_full

# calculate speed (approx.) in kmph
def calculate_speed(data_full):
    data_full['speed'] = (data_full['distance'] / data_full['duration']) * 3.6
    return data_full

# split data into modes
def split_data(mode, data_full):
    data_mode = data_full.loc[data_full['vehicleType'] == mode].copy()
    return data_mode   
    
# values to clean data
def get_values_to_clean_data(mode):
    values = {}
    # 388972800s equals 1 day
    if (mode == 'car'):
        values = {'duration_high': 388972800, 'duration_low': 60, 'distance_high': 35000, 'distance_low': 100, 
                  'speed_high': 130, 'speed_low': 0}
    if (mode == 'bicycle'):
        values = {'duration_high': 388972800, 'duration_low': 60, 'distance_high': 35000, 'distance_low': 100, 
                  'speed_high': 25, 'speed_low': 0}
    if (mode == 'scooter'):
        values = {'duration_high': 388972800, 'duration_low': 60, 'distance_high': 35000, 'distance_low': 100, 
                  'speed_high': 25, 'speed_low': 0}
    return values

# clean data based on mode and selected variables
def clean_data(mode, data_mode):
    # get values of mode
    values = get_values_to_clean_data(mode)
    # delete conditions based on values
    delete_conditions = [(data_mode['fuel_end']>data_mode['fuel_start']) | 
                  ((data_mode['distance']<values['distance_low'])|(data_mode['distance']>values['distance_high'])) |
                  ((data_mode['duration']<values['duration_low'])|(data_mode['duration']>values['duration_high'])) |
                  ((data_mode['speed']<values['speed_low'])|(data_mode['speed']>values['speed_high']))]
    # clean data based on conditions
    #for condition in delete_conditions:
    trips_to_delete = data_mode[delete_conditions[0]].index
    data_mode.drop(trips_to_delete , inplace=True)
    return data_mode

# prepare data
def prepare_data(data_mode):
    # drop unneeded variables
    data_mode.drop(['city', 'model', 'datetime_start', 'datetime_end', 'fuel_start', 'fuel_end', 
                    'price_driving_start', 'price_driving_end', 'price_promo_start', 'price_promo_end',
                    'missing_file_number'], axis = 1, inplace = True)
    # calculate start and end coordinates, round due to GPS accuracy
    data_mode["coordinates_start"] = list(zip(data_mode["latitude_start"].round(4), data_mode["longitude_start"].round(4)))
    data_mode["coordinates_end"] = list(zip(data_mode["latitude_end"].round(4), data_mode["longitude_end"].round(4)))
    # convert date
    data_mode['date_start'] = pd.to_datetime(data_mode['date_start'], format='%Y%m%d')
    data_mode['date_end'] = pd.to_datetime(data_mode['date_end'], format='%Y%m%d')
    # create columns year, month (1-12), weekday (0-6), hour (0-23) and weekend (binary identifier) based on date_start
    data_mode['year'] = data_mode['date_start'].apply(lambda x: x.year)
    data_mode['month'] = data_mode['date_start'].apply(lambda x: x.month)
    data_mode["weekday"] = data_mode["date_start"].apply(lambda x: x.weekday())
    data_mode["hour"] = (data_mode["time_start"] / 100).apply(np.floor).astype('int32')
    data_mode['weekend'] = np.where(data_mode['weekday'] > 4, 1, 0)
    # string switch: kick scooter into scooter (easier referencing)
    data_mode['vehicleType'] = np.where(data_mode['vehicleType'] == 'kick scooter', 'scooter', data_mode['vehicleType'])
    # arange order of columns
    data_mode = data_mode[['id', 'provider', 'vehicleType',  
                           'date_start', 'time_start', 'date_end', 'time_end', 
                           'year', 'month', 'weekday', 'hour', 'weekend', 
                           'longitude_start', 'latitude_start', 'longitude_end', 'latitude_end', 
                           'coordinates_start', 'coordinates_end', 
                           'distance', 'duration', 'speed']]
    # sort rows by date_start, time_start, time_end and id
    data_mode.sort_values(by = ['date_start', 'time_start', 'time_end', 'id'], ascending = True, inplace = True)
    # reset index
    data_mode = data_mode.reset_index(drop=True)
    return data_mode

# save data by mode and months
def save_prepared_data(months, data_prepared):
    for month in months:
        # extract month, year and mode
        only_month = int(month[4:])
        only_year = int(month[:4])
        mode_name_file = data_prepared['vehicleType'].iloc[0]
        # create dataframe and save it as csv file
        split_cond = [(data_prepared['month'] == only_month) & (data_prepared['year'] == only_year)]
        data_prepared[split_cond[0]].to_csv('Data/Modes/data_prepared_{}_{}.csv'.format(mode_name_file, month), index = False)

# save data by mode only
def save_prepared_data_full(data_prepared):
    mode_name_file = data_prepared['vehicleType'].iloc[0]
    # create dataframe and save it as csv file
    data_prepared.to_csv('Data/Modes/data_prepared_full_{}.csv'.format(mode_name_file), index = False)

### Execution

In [3]:
# define months for preparation (YYYYMM) - WARNING: too many months take a lot of RAM
months = ['201911', '201912', '202001', '202002']

In [4]:
# read in desired data by selected months and calculate speed to prepare for cleaning
data_full = calculate_speed(load_data(months))

#### `Free-floating car data`

In [5]:
# car
# split data to select desired mode
data_car = split_data('car', data_full)
# clean mode data
data_car = clean_data('car', data_car)
# prepare mode data
data_car = prepare_data(data_car)

In [6]:
data_car.tail()

,id,provider,vehicleType,date_start,time_start,date_end,time_end,year,month,weekday,...,weekend,longitude_start,latitude_start,longitude_end,latitude_end,coordinates_start,coordinates_end,distance,duration,speed
329140,ZnJlZWZsb2F0OjpjYXIyZ286OldNRTQ1MzM0MjFLMzgxOTk5,car2go,car,2020-02-29,2335,2020-02-29,2350,2020,2,5,...,1,6.95709,50.95142,6.95173,50.92336,"(50.9514, 6.9571)","(50.9234, 6.9517)",3142.658033,900.0,12.570632
329141,ZnJlZWZsb2F0OjpjYXIyZ286OldNRTQ1MzM0MjFLNDA5ODYz,car2go,car,2020-02-29,2335,2020-02-29,2350,2020,2,5,...,1,6.94209,50.95802,6.90843,50.92968,"(50.958, 6.9421)","(50.9297, 6.9084)",3935.988774,900.0,15.743955
329142,ZnJlZWZsb2F0OjpjYXIyZ286OldNRTQ1MzM0MjFLMzE5Nzg5,car2go,car,2020-02-29,2335,2020-02-29,2355,2020,2,5,...,1,6.96130,50.93441,6.99355,50.92724,"(50.9344, 6.9613)","(50.9272, 6.9936)",2396.629686,1202.0,7.177926
329143,ZnJlZWZsb2F0OjpjYXIyZ286OldNRTQ1MzM0MjFLMzIwMjgx,car2go,car,2020-02-29,2335,2020-02-29,2355,2020,2,5,...,1,6.92243,50.93069,6.91377,50.95217,"(50.9307, 6.9224)","(50.9522, 6.9138)",2464.337297,1202.0,7.380711
329144,ZnJlZWZsb2F0OjpjYXIyZ286OldNV1hNNTEwMTAySDIxODA0,car2go,car,2020-02-29,2340,2020-02-29,2355,2020,2,5,...,1,6.91724,50.95494,6.93603,50.94263,"(50.9549, 6.9172)","(50.9426, 6.936)",1899.040824,901.0,7.587732


#### `Free-floating bicycle data`

In [7]:
# bicycle
# split data to select desired mode
data_bicycle = split_data('bicycle', data_full)
# clean mode data
data_bicycle = clean_data('bicycle', data_bicycle)
# prepare mode data
data_bicycle = prepare_data(data_bicycle)

In [8]:
data_bicycle.head()

,id,provider,vehicleType,date_start,time_start,date_end,time_end,year,month,weekday,...,weekend,longitude_start,latitude_start,longitude_end,latitude_end,coordinates_start,coordinates_end,distance,duration,speed
0,YmlrZTo6Zm9yZHBhc3M6OjE3OTk5Mw==,fordpass,bicycle,2019-11-05,1055,2019-11-18,1300,2019,11,1,...,0,6.903943,50.915005,7.013771,50.940212,"(50.915, 6.9039)","(50.9402, 7.0138)",8191.946846,1130686.0,0.026082
1,YmlrZTo6Zm9yZHBhc3M6OjE3OTgyNA==,fordpass,bicycle,2019-11-05,1100,2019-11-05,1210,2019,11,1,...,0,7.007322,50.960327,7.006174,50.962013,"(50.9603, 7.0073)","(50.962, 7.0062)",203.996496,4201.0,0.174813
2,YmlrZTo6Zm9yZHBhc3M6OjE4MzQyMw==,fordpass,bicycle,2019-11-05,1100,2019-11-05,1210,2019,11,1,...,0,6.897302,50.924103,6.939653,50.934097,"(50.9241, 6.8973)","(50.9341, 6.9397)",3169.377086,4201.0,2.715962
3,YmlrZTo6Zm9yZHBhc3M6OjE2NjM4MQ==,fordpass,bicycle,2019-11-05,1100,2019-11-05,1215,2019,11,1,...,0,6.949904,50.936813,6.943760,50.937740,"(50.9368, 6.9499)","(50.9377, 6.9438)",442.662216,4498.0,0.354287
4,YmlrZTo6Zm9yZHBhc3M6OjE2NjcwMg==,fordpass,bicycle,2019-11-05,1100,2019-11-05,1215,2019,11,1,...,0,6.940404,50.929024,6.932660,50.927208,"(50.929, 6.9404)","(50.9272, 6.9327)",579.091061,4498.0,0.463479


#### `Free-floating scooter data`

In [9]:
# kick scooter
# split data to select desired mode
data_scooter = split_data('kick scooter', data_full)
# clean mode data
data_scooter = clean_data('scooter', data_scooter)
# prepare mode data
data_scooter = prepare_data(data_scooter)

In [10]:
data_scooter.head()

,id,provider,vehicleType,date_start,time_start,date_end,time_end,year,month,weekday,...,weekend,longitude_start,latitude_start,longitude_end,latitude_end,coordinates_start,coordinates_end,distance,duration,speed
0,YmlrZTo6bGltZTo6WFhYLU1XQQ==,lime,scooter,2019-11-05,1055,2019-11-05,1100,2019,11,1,...,0,6.959900,50.937970,6.938700,50.926564,"(50.938, 6.9599)","(50.9266, 6.9387)",1953.412266,284.0,24.761564
1,YmlrZTo6bGltZTo6WFhYLUFGVg==,lime,scooter,2019-11-05,1055,2019-11-05,1100,2019,11,1,...,0,6.945491,50.947803,6.944429,50.948414,"(50.9478, 6.9455)","(50.9484, 6.9444)",100.752583,284.0,1.277145
2,YmlrZTo6bGltZTo6WFhYLUJDSw==,lime,scooter,2019-11-05,1055,2019-11-05,1100,2019,11,1,...,0,6.942667,50.944721,6.940029,50.940649,"(50.9447, 6.9427)","(50.9406, 6.94)",489.057192,284.0,6.199317
3,YmlrZTo6bGltZTo6WFhYLURHTA==,lime,scooter,2019-11-05,1055,2019-11-05,1100,2019,11,1,...,0,6.930377,50.939279,6.942701,50.928754,"(50.9393, 6.9304)","(50.9288, 6.9427)",1454.482344,284.0,18.437100
4,YmlrZTo6bGltZTo6WFhYLUZERQ==,lime,scooter,2019-11-05,1055,2019-11-05,1100,2019,11,1,...,0,6.923084,50.936629,6.942513,50.928522,"(50.9366, 6.9231)","(50.9285, 6.9425)",1632.937540,284.0,20.699208


### Save data sets

In [11]:
# save prepared data sets based on selected months
# split data by month to avoid file size > 100 MB due to GitHub rules
# set True if saving is desired
save_data = False
if (save_data):
    save_prepared_data(months, data_car)
    save_prepared_data(months, data_bicycle)
    save_prepared_data(months, data_scooter)

In [12]:
# save prepared data sets in one file per mode
# set True if saving is desired
save_data_full = False
if (save_data_full):
    save_prepared_data_full(data_car)
    save_prepared_data_full(data_bicycle)
    save_prepared_data_full(data_scooter)

### Test

In [13]:
data_bicycle.isnull().values.any()

False

In [14]:
len(data_car[(data_car['weekday'] == 0) & (data_car['hour'] == 2)])

296

In [15]:
type(data_full["time_start"])

pandas.core.series.Series

In [16]:
data_car['provider'].unique()

array(['drivenow', 'car2go', 'driveby'], dtype=object)